The following Script calls the dbpedia api according the following links <br>
http://lookup.dbpedia.org/api/search/KeywordSearch?QueryClass=place&QueryString=berlin

In [2]:
import json
import xmltodict
import pandas as pd
import requests
import random
import csv
import time

In [3]:
base_url = "http://lookup.dbpedia.org/api/search/KeywordSearch?QueryString="

In [4]:
def callFascetGenerationAPI(url):
    response = requests.post(url)

    if response.status_code !=200:
        print("Error status code:" + str(response.status_code))
    
    #print(response)
    #return only the fasctes
    return (response)

In [5]:
#def extractALLCategories(res_content):
#    categories = []
#    for res_el in res_content['ArrayOfResults']['Result']:
#        if res_el['Categories'] != None:
#            if len(res_el['Categories']['Category']) > 1:
#                for url_el in res_el['Categories']['Category']:
#                    cat_url = url_el['URI']
#                    categories.append(cat_url.replace('http://dbpedia.org/resource/Category:','').replace('_',' '))
#            else:
#                cat_url = res_el['Categories']['Category']['URI']
#                categories.append(cat_url.replace('http://dbpedia.org/resource/Category:','').replace('_',' '))
#    return categories

#Prints first 3 categories of the first X results
def extractXCategories(res_content, num_categories):
    categories = []
    if isinstance(res_content['ArrayOfResults']['Result'], list):
        for (i, res_el) in enumerate(res_content['ArrayOfResults']['Result']):
            if res_el['Categories'] != None:
                if len(res_el['Categories']['Category']) > 1:
                    for (idx, url_el) in enumerate(res_el['Categories']['Category']):
                        categories.append(url_el['URI'].replace('http://dbpedia.org/resource/Category:','').replace('_',' '))
                        if idx > 0: #0,1,2 => get first 3
                            break
                else:
                    cat_url = res_el['Categories']['Category']['URI']
                    categories.append(cat_url.replace('http://dbpedia.org/resource/Category:','').replace('_',' '))
    else: #if there is only one elemtn 
        res_el = res_content['ArrayOfResults']['Result']
        if res_el['Categories'] != None:
                if len(res_el['Categories']['Category']) > 1:
                    for (idx, url_el) in enumerate(res_el['Categories']['Category']):
                        categories.append(url_el['URI'].replace('http://dbpedia.org/resource/Category:','').replace('_',' '))
                        if idx > 0: #0,1,2 => get first 3
                            break
                else:
                    cat_url = res_el['Categories']['Category']['URI']
                    categories.append(cat_url.replace('http://dbpedia.org/resource/Category:','').replace('_',' '))
    return (categories[0:num_categories])

In [6]:
#def extractAllClasses(res_content):
#    classes = []
#    for (i, res_el) in enumerate(res_content['ArrayOfResults']['Result']):
#        if res_el['Classes'] != None:
#            if isinstance(res_el['Classes']['Class'], list): #check if it is a list of elements or a dict of one element
#                for (idx, class_el) in enumerate(res_el['Classes']['Class']):
#                    classes.append(class_el['Label'])
#            else:
#                classes.append(res_el['Classes']['Class']['Label'])
#    return classes

def extractXClasses(res_content, num_classes):
    classes = []
    if isinstance(res_content['ArrayOfResults']['Result'], list):
        for (i, res_el) in enumerate(res_content['ArrayOfResults']['Result']):
            if res_el['Classes'] != None:
                if isinstance(res_el['Classes']['Class'], list): #check if it is a list of elements or a dict of one element
                    for (idx, class_el) in enumerate(res_el['Classes']['Class']):
                        classes.append(class_el['Label'])
                        if idx > 0: #0,1,2 => get first 3
                            break
                else:
                    classes.append(res_el['Classes']['Class']['Label'])
            if i > 20:
                break
    else: #if there is only one elemtn 
        res_el = res_content['ArrayOfResults']['Result']
        if res_el['Classes'] != None:
                if isinstance(res_el['Classes']['Class'], list): #check if it is a list of elements or a dict of one element
                    for (idx, class_el) in enumerate(res_el['Classes']['Class']):
                        classes.append(class_el['Label'])
                        if idx > 0: #0,1,2 => get first 3
                            break
                else:
                    classes.append(res_el['Classes']['Class']['Label'])
    return classes[0:num_classes]

In [8]:
df = pd.read_csv("../Data/DF_MIMICS_ConstructedFascets_v2_6.csv")


In [9]:
df

,Unnamed: 0.1,Unnamed: 0,query,fascets,target,snippet,fascets_5,fascets_12,fascets_20
0,0,0,rimouski,"climate , economy , municipal council , history",What do you want to know about the place?,"[""Rimouski - Wikipedia Rimouski (/\u02ccr\u026...","rimouski weather , canada , weather , events ,...","rimouski weather , canada , weather , events ,...","rimouski weather , canada , weather , events ,..."
1,1,1,ohe,"office of higher education , office of health ...","Which ""ohe"" do you mean?","[""Minnesota Office of Higher Education The Min...","office of health economics , of canada , offic...","office of health economics , of canada , offic...","office of health economics , of canada , offic..."
2,2,2,ballet folklorico de mexico,"ballet folklorico de mexico cd , ballet folklo...",What format are you looking for?,"[""Ballet Folkl\u00f3rico de M\u00e9xico de Ama...","ballet folklorico de mexico dvd , vhs , hotels...","ballet folklorico de mexico dvd , vhs , hotels...","ballet folklorico de mexico dvd , vhs , hotels..."
3,3,3,ubac,"ubuntu business advisory and consulting , user...","Which ""ubac"" do you mean?","[""UBAC \u2013 Business Advisory & Consulting W...","ubuntu business advisory and consulting , dod ...","ubuntu business advisory and consulting , dod ...","ubuntu business advisory and consulting , dod ..."
4,4,4,book of ezra,"book of ezra summary , who wrote book of ezra ...",What would you like to know about this book?,"[""Book of Ezra - Wikipedia The Book of Ezra is...","who wrote book of ezra , kjv , nehemiah , cast...","who wrote book of ezra , kjv , nehemiah , cast...","who wrote book of ezra , kjv , nehemiah , cast..."
...,...,...,...,...,...,...,...,...,...
19046,19046,19046,karnataka legislative council,"constituencies , term , history",What do you want to know about karnataka legis...,"[""Karnataka Legislative Council - Wikipedia Th...","karnataka legislative council logo , usa , ele...","karnataka legislative council logo , usa , ele...","karnataka legislative council logo , usa , ele..."
19047,19047,19047,bsps,"british show pony society , bachelor of scienc...","Which ""bsps"" do you mean?","[""British Show Pony Society A Company limited ...","british show pony society , canada , british s...","british show pony society , canada , british s...","british show pony society , canada , british s..."
19048,19048,19048,canon pro-100,"manual , driver",What information are you looking for?,"[""Canon U.S.A., Inc. | PIXMA PRO-100 Canon off...","driver , ebay , printer , manual , amazon","driver , ebay , printer , manual , amazon , ac...","driver , ebay , printer , manual , amazon , ac..."
19049,19049,19049,rv max,"shops photo , local map , business information",What do you want to know about this business?,"[""RV Max\u00ae - Loomis & Rocklin, CA - Where ...","rv max service , coupon , keystone , manual , ...","rv max service , coupon , keystone , manual , ...","rv max service , coupon , keystone , manual , ..."


In [10]:
#df = df[0:2381]

In [11]:
allCat = []
allCla = []

for idx, item in df.iterrows():
    print(idx)
    try:
        categories = ""
        classes = ""
        #print("------------------------------------")
        for fascet in item['fascets_20'].split(' , '):
            #print(fascet)
            url = base_url + fascet
            response = callFascetGenerationAPI(url)

            #time.sleep(0.5)

            res_content = xmltodict.parse(response.content)

            if res_content['ArrayOfResults'] == None:
                categories = categories + " | " + fascet
                classes = classes + " | " + fascet
            else:
                categories = categories + " | " + fascet + " = " + (' , '.join(extractXCategories(res_content,5)))
                classes = classes + " | " + fascet + " = " + (' , '.join(extractXClasses(res_content,5)))

        allCat.append(categories)
        allCla.append(classes)
        
    except:
        allCat.append(" ")
        allCla.append(" ")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
Error status code:500
140
141
142
143
144
145
146
Error status code:500
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
26

1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
Error status code:500
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
202

In [12]:
allCat

[" | rimouski weather | canada = 1867 establishments in Canada , English-speaking countries and territories , 1867 establishments in Canada , Provinces of Canada , 1867 establishments in Canada | weather = Cities and towns in Quebec , Hudson's Bay Company trading posts , Cities and towns in Lombardy , Populated places established in the 6th century BC , American television networks | events = Television terminology , Sociology of knowledge , K-1 events , K-1 MAX events , Television terminology | rimouski municipality = Census divisions of Quebec , Regional county municipalities in Bas-Saint-Laurent , Census divisions of Quebec , Regional county municipalities in Bas-Saint-Laurent , Cities and towns in Quebec | cambodia = 1993 establishments in Asia , Countries in Asia , Country subdivisions in Asia , Subdivisions of Cambodia , Country subdivisions in Asia | things to do in = Virgin Records singles , 1990 singles , Officers of the Order of the British Empire , 1960 deaths , Living peopl

In [13]:
try:
    df['DBpedia_classes'] = allCla
    df['DBpedia_categories'] = allCat
except:
    df_cla = pd.DataFrame(allCla)
    df_cla.to_csv("../Data/DBpedia/DF_MIMICS_Full_constructed_DBpediaClasses_P1.csv")
    df_cat = pd.DataFrame(allCat)
    df_cat.to_csv("../Data/DBpedia/DF_MIMICS_Full_constructed_DBpediaCategories_P1.csv")

C:\Users\DFINGE~1\AppData\Local\Temp/ipykernel_4548/2001257866.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DBpedia_classes'] = allCla
C:\Users\DFINGE~1\AppData\Local\Temp/ipykernel_4548/2001257866.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DBpedia_categories'] = allCat


In [14]:
df.to_csv("../Data/DBpedia/DF_MIMICS_DBpedia_v3_3.csv")